In [1]:
# Dependencies and Setup
import base64
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import base64
from requests.auth import HTTPBasicAuth
import scipy.stats as st

# Import Spotify API key
from api_keys import spotify_client_id, spotify_client_secret

# Spotify Playlists Data Output File (CSV)
spotify_playlists_csv = "../datasets/spotify_playlists_data.csv"

In [2]:
# Create playlist dataframe
play_df = pd.read_csv("sample.csv")
play_df.head()

,Name,Link,List ID,Unnamed: 3,Unnamed: 4,Unnamed: 5
0,Todays Top Hits,https://open.spotify.com/playlist/37i9dQZF1DXc...,37i9dQZF1DXcBWIGoYBM5M,NaN,NaN,NaN
1,RapCaviar,https://open.spotify.com/playlist/37i9dQZF1DX0...,37i9dQZF1DX0XUsuxWHRQd,NaN,NaN,NaN
2,Get Turnt,https://open.spotify.com/playlist/37i9dQZF1DWY...,37i9dQZF1DWY4xHQp97fN6,NaN,NaN,NaN
3,Hot Country,https://open.spotify.com/playlist/37i9dQZF1DX1...,37i9dQZF1DX1lVhptIYRda,NaN,NaN,NaN
4,Chill Hits,https://open.spotify.com/playlist/37i9dQZF1DX4...,37i9dQZF1DX4WYpdgoIcn6,NaN,NaN,NaN


In [3]:
"""
Code to manage Spotify's Client Credentials Flow as described here:
https://developer.spotify.com/documentation/general/guides/authorization-guide/#client-credentials-flow
"""
# Build the Spotify Client ID and Secret combination
spotify_client_id_secret = spotify_client_id + ":" + spotify_client_secret
# Generate the base64 encoded string that contains the client ID and client secret key
base64_encoding = base64.b64encode(spotify_client_id_secret.encode()).decode()

# Build the Spotify API URL to get the Bearer token
spotify_token_url = 'https://accounts.spotify.com/api/token'

# Build the payload, i.e. the request body parameters that need to be passed to the token API URL
payload = 'grant_type=client_credentials'
# Build the header for Content-Type and Basic Auth containing the base64 encoded string so we can generate the Spotify API token
token_api_headers = { 'Content-Type': 'application/x-www-form-urlencoded', 'Authorization': 'Basic %s' % base64_encoding }

# Call the token API URL and save as a JSON response object
spotify_token_response = requests.post(spotify_token_url, headers=token_api_headers, data=payload).json()
# Retrieve the Bearer token from the response object
spotify_access_token = spotify_token_response['access_token']

In [4]:
"""
Code to retrieve a Spotify Playlist's Tracks and related data:
https://developer.spotify.com/documentation/web-api/reference/playlists/get-playlists-tracks/
"""
# Sample Playlist ID
ids = play_df["List ID"]

# Initialize lists to save playlist data
track_added_at = []
track_names = []
track_popularity = []
track_durations = []
artists = []
albums = []
album_types = []
release_dates = []
num_available_markets = []

#loops for creating dataframe columns
for id in ids:
    try:
        # Build Spotify's Get Playlist's Tracks URL
        spotify_playlists_url = f'https://api.spotify.com/v1/playlists/{id}/tracks'
        # Build the header for Bearer token containing the base64 encoded string so we can retrieve the playlist's tracks 
        playlist_api_headers = { 'Authorization': 'Bearer %s' % spotify_access_token }
        # Call the Get Playlist's Tracks API URL and save as a JSON response object
        spotify_playlists_response = requests.get(spotify_playlists_url, headers=playlist_api_headers, data={}).json()

        # Get total tracks in playlist
        total_tracks = spotify_playlists_response['total']

        # Playlist returns a track object which we will parse to retrieve all the track, album and artist data
        for item in range(len(spotify_playlists_response['items'])):

            # Get the date and time the track was added to the playlist 
            track_added_at.append(spotify_playlists_response['items'][item]['added_at'])
            # Get the name of the track
            track_names.append(spotify_playlists_response['items'][item]['track']['name'])
            # Get the popularity score of the track
            track_popularity.append(spotify_playlists_response['items'][item]['track']['popularity'])
            # Get the track duration (song length) in milliseconds
            track_durations.append(spotify_playlists_response['items'][item]['track']['duration_ms'])
            # Get the number of markets that the track is available in
            num_available_markets.append(len(spotify_playlists_response['items'][item]['track']['available_markets']))
            # Get the album name
            albums.append(spotify_playlists_response['items'][item]['track']['album']['name'])
            # Get the album type: single, album or compilation
            album_types.append(spotify_playlists_response['items'][item]['track']['album']['album_type'])
            # Get the album release date
            release_dates.append(spotify_playlists_response['items'][item]['track']['album']['release_date'])

            # Each track could be associated with one or more artists
            # Parse through each artist object to retrieve their names 
            artist_name = ''
            artists_count = len(spotify_playlists_response['items'][item]['track']['artists'])
            for artist in range(artists_count):
                artist_name = artist_name + ' ' + spotify_playlists_response['items'][item]['track']['artists'][artist]['name']

            artists.append(artist_name)
    except Exception:
        print("Nope")
# Save the retrieved playlist's track data into a Data Frame
playlist_data_df = pd.DataFrame( {'Track Name': track_names, 'Track Added At': track_added_at, 'Track Popularity': track_popularity, 'Track Duration': track_durations, 'Available Markets': num_available_markets, 'Album': albums, 'Album Type': album_types, 'Release Date': release_dates, 'Artist': artists } )

# Display the playlist's tracks data frame
playlist_data_df

Nope


KeyboardInterrupt: 